In [52]:
#A.Web Scraping

In [53]:
import requests
from bs4 import BeautifulSoup
import time
import csv

In [54]:
BASE_URL = "http://books.toscrape.com/catalogue/page-{}.html"

In [55]:
def get_soup(url, retries=3):
    for i in range(retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except Exception:
            if i == retries - 1:
                print(f"Failed: {url}")
                return None
            time.sleep(2)

books_data = []

In [56]:
#loop to get books
#get each book detail 

In [57]:
#looped through each book catalogue
#because to get category for each book
#took more time than usual

for page in range(1, 6):
    url = BASE_URL.format(page)
    soup = get_soup(url)

    if not soup:
        continue

    books = soup.find_all('article', class_='product_pod')

    for book in books:
        try:
            title = book.h3.a['title']
            price = book.find('p', class_='price_color').text
            rating = book.p['class'][1]
            availability = book.find('p', class_='instock availability').text.strip()

            #Get book detail page
            book_url = book.h3.a['href']
            book_url = "http://books.toscrape.com/catalogue/" + book_url

            book_soup = get_soup(book_url)

            category = "Unknown"
            try:
                breadcrumb = book_soup.find('ul', class_='breadcrumb')
                category = breadcrumb.find_all('li')[2].text.strip()
            except:
                pass

            books_data.append([title, price, rating, availability, category])

        except Exception:
            continue

    time.sleep(2)

In [64]:
# Save to CSV
with open('books_uncleaned.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Price', 'Rating', 'Availability', 'Category'])
    writer.writerows(books_data)